In [5]:
from scipy import optimize
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
data = pd.read_csv("Brightest Stars 2019-9-07 08h30m10s Vmag-4 Alt-20.csv")

In [3]:
data

,x,y,alt,az,alt_err,az_err,sample_size,star_name,ra,dec
0,442.0,476.0,35.300056,89.672226,0.049080,0.075814,2.0,9Alp CMa,06h45m08.9s,-16d42m58s
1,568.0,642.0,45.850794,136.001391,0.000000,0.125174,2.0,Alp Car,06h23m57.1s,-52d41m45s
2,539.0,386.0,49.749357,64.048453,0.079946,0.096327,3.0,19Bet Ori,05h14m32.3s,-08d12m06s
3,780.0,620.0,60.498814,200.249893,0.052536,0.051563,2.0,Alp Eri,01h37m42.9s,-57d14m12s
4,471.0,314.0,31.800362,57.309314,0.050901,0.006475,2.0,58Alp Ori,05h55m10.3s,+07d24m25s
...,...,...,...,...,...,...,...,...,...,...
160,954.0,626.0,37.601460,236.480414,0.051452,0.013435,2.0,Del1Gru,22h29m16.2s,-43d29m44s
161,974.0,476.0,42.403260,269.927522,0.048287,0.014563,2.0,98 Aqr,23h22m58.2s,-20d06m02s
162,459.0,401.0,36.549784,73.923677,0.082329,0.090985,3.0,5Gam Mon,06h14m51.3s,-06d16m29s
163,609.0,727.0,36.885030,154.520311,0.186724,0.095457,3.0,Del Vol,07h16m49.8s,-67d57m26s


In [4]:
## x, y = 836, 592
data.query("x==780") 

,x,y,alt,az,alt_err,az_err,sample_size,star_name,ra,dec
3,780.0,620.0,60.498814,200.249893,0.052536,0.051563,2.0,Alp Eri,01h37m42.9s,-57d14m12s
49,780.0,136.0,23.802649,350.837787,0.049968,0.020612,2.0,4Bet Tri,02h09m32.6s,+34d59m14s


In [5]:
X0, Y0, a0 = float(757), float(472), float(0)

In [6]:
def construct_radio(C, A, F):
    def r(x, y):
        return C * ( np.sqrt( (x-X0)**2 + (y-Y0)**2 ) + A*(y-X0)*np.cos(F-a0) - A*(x-X0)*np.sin(F-a0) )
    return r

In [7]:
def construct_u(C, A, F, V, S, D, P, Q):
    def u(x, y):
        r = construct_radio(C, A, F)
        return V*r(x, y) + S*(np.e**(D*r(x,y)) - 1) + P*(np.e**(Q*r(x,y)**2) - 1) 
    return u

In [8]:
def construct_b(E):
    def b(x, y):
        return a0 - E + np.arctan2((y - Y0),(x - X0))
    return b

In [9]:
def construct_altura(C, A, F, V, S, D, P, Q, E, ep):
    def z(x, y):
        u = construct_u(C, A, F, V, S, D, P, Q)
        b = construct_b(E)
        return np.arccos( np.cos(u(x,y))*np.cos(ep) - np.sin(u(x,y))*np.sin(ep)*np.cos(b(x,y)) )
    return z

def construct_altura_deg(C, A, F, V, S, D, P, Q, E, ep):
    def z(x, y):
        u = construct_u(C, A, F, V, S, D, P, Q)
        b = construct_b(E)
        return 90 - np.rad2deg( np.arccos( np.cos(u(x,y))*np.cos(ep) - np.sin(u(x,y))*np.sin(ep)*np.cos(b(x,y)) ) )
    return z

In [10]:
def construct_azimuth(C, A, F, V, S, D, P, Q, E, ep):
    def az(x, y):
        u = construct_u(C, A, F, V, S, D, P, Q)
        b = construct_b(E)
        z = construct_altura(C, A, F, V, S, D, P, Q, E, ep)
        return np.arcsin( np.sin(b(x,y))*np.sin(u(x,y))/np.sin(z(x,y)) ) + E
    return az

def construct_azimuth_deg(C, A, F, V, S, D, P, Q, E, ep):
    def az(x, y):
        u = construct_u(C, A, F, V, S, D, P, Q)
        b = construct_b(E)
        z = construct_altura(C, A, F, V, S, D, P, Q, E, ep)
        return np.rad2deg( np.arcsin( np.sin(b(x,y))*np.sin(u(x,y))/np.sin(z(x,y)) ) + E ) + 90
    return az

In [11]:
def plot_plano(plano):
    x = np.linspace(0, 1200, 100)
    y = np.linspace(0, 900, 100)
    X, Y = np.meshgrid(x, y)
    Z = plano(X, Y)
    fig,ax=plt.subplots(1,1)
    cp = ax.contourf(X, Y, Z, 10)
    fig.colorbar(cp) # Add a colorbar to a plot
    ax.set_title('Toda una linea que minimiza y=mx+n')
    plt.gca().invert_yaxis()
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.show()
    

In [12]:
def plot_altura(vector):
    plano = construct_altura_deg(*vector)
    plot_plano(plano)
    
def plot_azimuth(vector):
    plano = construct_azimuth_deg(*vector)
    plot_plano(plano)

In [13]:
C, A, F = 0.07, 0.01, 1
V, S, D, P, Q = 0.03, 0.007, 0.09, 2.2*10**-6, 0.006
E, ep = 0, 0
parametros_inciales = [C, A, F, V, S, D, P, Q, E, ep]
result1 = np.array([ 4.05411155*10**-2, -5.13000215*10**-2,  1.18306086,  8.18813843*10**-2,
       -3.63276373*10**-3, -1.22830557,  1.09298238*10**-5,  1.29796815*10**-2,
        2.09851909*10**-3,  5.53361549*10**-2])
result2 = np.array([ 2.90396335*10**-2, -1.36275018*10**-2, -2.42156457*10**-1,  1.14623453*10**-1,
       -6.22511285*10**-3, -4.72148386,  4.53040216*10**-5,  1.57578545*10**-2,
       -3.35189511*10**-2,  1.03652889*10**-1])
result3 = np.array([ 2.87039016*10**-2, -5.47881381*10**-3, -2.49778009*10**-1,  1.14866070*10**-1,
       -2.19097809*10**-2, -4.58387917*10**-1,  2.62567843*10**-4,  2.39744298*10**-2,
       -1.06146519*10**-1,  1.02486171*10**-1])
result4 = np.array([ 0.02863326, -0.00550686, -0.30656714,  0.11274438, -0.03916933,
       -0.29909259,  0.00547779,  0.01104806, -0.10624677,  0.1027069 ])
result5 = np.array([ 0.02896625, -0.00554929, -0.3355378 ,  0.11107163, -0.04046421,
       -0.29994853,  0.00962758,  0.00862338, -0.10597777,  0.10286461])
result6 = np.array([ 4.88504077*10**-2,  2.02995372*10**-2,  3.51610379*10**-1,  6.82171007*10**-2,
         3.10608468*10**-3, -5.75141386*10**-5,  4.26865327*10**-6, -4.15889519*10**-3,
         2.21040767*10**-3,  1.47892726*10**-3])
result7 = np.array([ 6.68386793*10**-2,  2.11643138*10**-2,  3.95994947*10**-1,  4.97793056*10**-2,
        -1.98285373*10**-4,  3.95323036*10**-2,  1.77167244*10**-6,  8.38067924*10**-3,
         4.35828992*10**-4,  2.52528618*10**-3])

In [14]:
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from datetime import datetime

# from astropy.utils.iers import conf
# conf.auto_max_age = None

In [15]:
EL_SAUCE = EarthLocation(lat=-30.4726064*u.deg, lon=-70.7653747*u.deg, height=789*u.m)
obs_time = "2019-9-10 08:30:10"
alt_az_frame = AltAz(obstime=Time(obs_time), location=EL_SAUCE)
c = SkyCoord("01h37m42.9s", "-57d14m12s", frame='icrs')
c_altaz = c.transform_to(alt_az_frame)
c_altaz.alt.value

59.526749222713995

In [25]:
plano0 = construct_altura_deg(*parametros_inciales)
plano1 = construct_altura_deg(*result1)
plano2 = construct_altura_deg(*result2)
plano3 = construct_altura_deg(*result3)
plano4 = construct_altura_deg(*result4)
plano5 = construct_altura_deg(*result5)
plano6 = construct_altura_deg(*result6)
plano7 = construct_altura_deg(*result7)
plano0(836, 592), plano1(836, 592), plano2(836, 592), plano3(836, 592), plano4(836, 592), plano5(836, 592), plano6(836, 592), plano7(836, 592)


(72.31984706934044,
 59.29059983755271,
 58.40263480001899,
 58.54830917428019,
 58.53510142137111,
 58.532329618461205,
 63.227633292137845,
 63.27396661677527)

In [44]:
## x, y = 849, 519
data.query("x==804 and y==550") 

,x,y,alt,az,alt_err,az_err,sample_size,star_name,ra,dec
76,804.0,550.0,69.352215,226.341282,0.0,0.0,1.0,Gam Phe,01h28m21.9s,-43d19m06s


In [26]:
EL_SAUCE = EarthLocation(lat=-30.4726064*u.deg, lon=-70.7653747*u.deg, height=789*u.m)
obs_time = "2019-9-10 08:30:10"
alt_az_frame = AltAz(obstime=Time(obs_time), location=EL_SAUCE)
c = SkyCoord("01h28m21.9s", "-43d19m06s", frame='icrs')
c_altaz = c.transform_to(alt_az_frame)

plano0 = construct_altura_deg(*parametros_inciales)
plano1 = construct_altura_deg(*result1)
plano2 = construct_altura_deg(*result2)
plano3 = construct_altura_deg(*result3)
plano4 = construct_altura_deg(*result4)
plano5 = construct_altura_deg(*result5)
plano6 = construct_altura_deg(*result6)
plano7 = construct_altura_deg(*result7)
c_altaz.alt.value, plano0(849, 519), plano1(849, 519), plano2(849, 519), plano3(849, 519), plano4(849, 519), plano5(849, 519), plano6(849, 519), plano7(849, 519)

(67.3718874759399,
 77.45948568073499,
 65.56745009537073,
 64.04865532943674,
 64.21869438708212,
 64.20986910172411,
 64.20097034547302,
 71.18785266684822,
 71.20903840638339)

In [47]:
## x, y = 598, 249
data.query("x==601 and y==244") 

,x,y,alt,az,alt_err,az_err,sample_size,star_name,ra,dec
104,601.0,244.0,39.146918,28.104693,0.0,0.0,1.0,54Gam Tau,04h19m47.6s,+15d37m39s


In [27]:
EL_SAUCE = EarthLocation(lat=-30.4726064*u.deg, lon=-70.7653747*u.deg, height=789*u.m)
obs_time = "2019-9-10 08:30:10"
alt_az_frame = AltAz(obstime=Time(obs_time), location=EL_SAUCE)
c = SkyCoord("04h19m47.6s", "+15d37m39s", frame='icrs')
c_altaz = c.transform_to(alt_az_frame)

plano0 = construct_altura_deg(*parametros_inciales)
plano1 = construct_altura_deg(*result1)
plano2 = construct_altura_deg(*result2)
plano3 = construct_altura_deg(*result3)
plano4 = construct_altura_deg(*result4)
plano5 = construct_altura_deg(*result5)
plano6 = construct_altura_deg(*result6)
plano7 = construct_altura_deg(*result7)
c_altaz.alt.value, plano0(598, 249), plano1(598, 249), plano2(598, 249), plano3(598, 249), plano4(598, 249), plano5(598, 249), plano6(598, 249), plano7(598, 249)

(40.23317715932928,
 55.38371020755151,
 39.0606147278997,
 39.14192892123026,
 39.13987898638099,
 39.159995035382664,
 39.16546477667036,
 39.39279164158845,
 39.52708163570737)

In [50]:
## x, y = 661,391
data.query("x==641 and y==393") 

,x,y,alt,az,alt_err,az_err,sample_size,star_name,ra,dec
47,641.0,393.0,66.849621,45.794764,0.08403,0.221295,3.0,34Gam Eri,03h58m01.8s,-13d30m31s


In [28]:
EL_SAUCE = EarthLocation(lat=-30.4726064*u.deg, lon=-70.7653747*u.deg, height=789*u.m)
obs_time = "2019-9-10 08:30:10"
alt_az_frame = AltAz(obstime=Time(obs_time), location=EL_SAUCE)
c = SkyCoord("03h58m01.8s", "-13d30m31s", frame='icrs')
c_altaz = c.transform_to(alt_az_frame)

plano0 = construct_altura_deg(*parametros_inciales)
plano1 = construct_altura_deg(*result1)
plano2 = construct_altura_deg(*result2)
plano3 = construct_altura_deg(*result3)
plano4 = construct_altura_deg(*result4)
plano5 = construct_altura_deg(*result5)
plano6 = construct_altura_deg(*result6)
plano7 = construct_altura_deg(*result7)
c_altaz.alt.value, plano0(661,391), plano1(661,391), plano2(661,391), plano3(661,391), plano4(661,391), plano5(661,391), plano6(661,391), plano7(661,391)

(68.70972794960927,
 74.55022774051567,
 67.7585057985618,
 68.76088241363638,
 68.51920139433852,
 68.5129601963193,
 68.51607943086739,
 67.28594498688986,
 67.38275867766123)

In [32]:
datos = pd.read_csv("Data.pixtab", sep=" ")
datos = datos.dropna()
datos.query("x==333 and y==338").alt.values[0]

9.727276402704566

In [5]:
from scipy import optimize, stats
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from datetime import datetime

In [6]:
"""
Funciones de Borovicka que
dependen de [C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0]
"""

def construct_radio(C, A, F, X0, Y0, a0):
    def r(x, y):
        return C * ( np.sqrt( (x-X0)**2 + (y-Y0)**2 ) + A*(y-X0)*np.cos(F-a0) - A*(x-X0)*np.sin(F-a0) )
    return r

def construct_u(C, A, F, V, S, D, P, Q, X0, Y0, a0):
    def u(x, y):
        r = construct_radio(C, A, F, X0, Y0, a0)
        return V*r(x, y) + S*(np.e**(D*r(x,y)) - 1) + P*(np.e**(Q*r(x,y)**2) - 1) 
    return u

def construct_b(E, X0, Y0, a0):
    def b(x, y):
        return a0 - E + np.arctan2((y - Y0),(x - X0))
    return b

def construct_altura(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
    def z(x, y):
        u = construct_u(C, A, F, V, S, D, P, Q, X0, Y0, a0)
        b = construct_b(E, X0, Y0, a0)
        return np.arccos( np.cos(u(x,y))*np.cos(ep) - np.sin(u(x,y))*np.sin(ep)*np.cos(b(x,y)) )
    return z

def construct_altura_deg(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
    def z(x, y):
        u = construct_u(C, A, F, V, S, D, P, Q, X0, Y0, a0)
        b = construct_b(E, X0, Y0, a0)
        return 90 - np.rad2deg( np.arccos( np.cos(u(x,y))*np.cos(ep) - np.sin(u(x,y))*np.sin(ep)*np.cos(b(x,y)) ) )
    return z

def construct_azimuth(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
    def az(x, y):
        u = construct_u(C, A, F, V, S, D, P, Q, X0, Y0, a0)
        b = construct_b(E, X0, Y0, a0)
        z = construct_altura(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0)
        return np.arcsin( np.sin(b(x,y))*np.sin(u(x,y))/np.sin(z(x,y)) ) + E
    return az

def construct_azimuth_deg(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
    def az(x, y):
        u = construct_u(C, A, F, V, S, D, P, Q, X0, Y0, a0)
        b = construct_b(E, X0, Y0, a0)
        z = construct_altura(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0)
        return np.rad2deg( np.arcsin( np.sin(b(x,y))*np.sin(u(x,y))/np.sin(z(x,y)) ) + E ) + 90
    return az

"""
Estos son constructores de una funcion de Xi**2
"""

def construct_alt_xi(x, y, z):
    def xi(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
        alt_teorico = construct_altura_deg(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0)
        return (z - alt_teorico(x, y))**2
    return xi

def construct_az_xi(x, y, z):
    def xi(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
        az_teorico = construct_azimuth_deg(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0)
        return (z - az_teorico(x, y))**2
    return xi

In [38]:
params = [0.04952671, 0.00310034, -1.04782185,
          0.0682171007, 0.00310608468, -5.75141386e-05, 4.26865327e-06, -0.00415889519,
          -4.82552710e-03, -0.0022711518934522366,
          7.28125200e+02, 4.78181845e+02, 7.73853421e-01]
altura_teorica = construct_altura_deg(*params)
azimut_teorico = construct_azimuth_deg(*params)

data = pd.read_csv("Brightest Stars 2019-9-07 08h30m10s Vmag-4 Alt-20.csv")
EL_SAUCE = EarthLocation(lat=-30.4726064*u.deg, lon=-70.7653747*u.deg, height=789*u.m)
obs_time = "2019-9-07 08:30:10"
alt_az_frame = AltAz(obstime=Time(obs_time), location=EL_SAUCE)

In [32]:
data.query("x==780 and y==620") 

,x,y,alt,az,alt_err,az_err,sample_size,star_name,ra,dec
3,780.0,620.0,60.498814,200.249893,0.052536,0.051563,2.0,Alp Eri,01h37m42.9s,-57d14m12s


In [40]:
c = SkyCoord("01h28m21.9s", "-43d19m06s", frame='icrs')
c_altaz = c.transform_to(alt_az_frame)
c_altaz.alt.value - altura_teorica(780, 620), c_altaz.az.value - azimut_teorico(780, 620)

(8.593293282850965, 71.40987664773976)

In [34]:
data.query("x==804 and y==550") 

,x,y,alt,az,alt_err,az_err,sample_size,star_name,ra,dec
76,804.0,550.0,69.352215,226.341282,0.0,0.0,1.0,Gam Phe,01h28m21.9s,-43d19m06s


In [41]:
c = SkyCoord("03h58m01.8s", "-13d30m31s", frame='icrs')
c_altaz = c.transform_to(alt_az_frame)
c_altaz.alt.value - altura_teorica(804, 550), c_altaz.az.value - azimut_teorico(804, 550)

(-2.7388745536655534, -132.08783026625957)

In [36]:
data.query("x==601 and y==244") 

,x,y,alt,az,alt_err,az_err,sample_size,star_name,ra,dec
104,601.0,244.0,39.146918,28.104693,0.0,0.0,1.0,54Gam Tau,04h19m47.6s,+15d37m39s


In [42]:
c = SkyCoord("04h19m47.6s", "+15d37m39s", frame='icrs')
c_altaz = c.transform_to(alt_az_frame)
c_altaz.alt.value - altura_teorica(601, 244), c_altaz.az.value - azimut_teorico(601, 244)

(0.6425793023145303, 12.363243855630348)

# Otro

In [8]:
def construct_radio(C, A, F, X0, Y0, a0):
    def r(x, y):
        return C * ( np.sqrt( (x-X0)**2 + (y-Y0)**2 ) + A*(y-X0)*np.cos(F-a0) - A*(x-X0)*np.sin(F-a0) )
    return r

def construct_u(C, A, F, V, S, D, P, Q, X0, Y0, a0):
    def u(x, y):
        r = construct_radio(C, A, F, X0, Y0, a0)
        return V*r(x, y) + S*(np.e**(D*r(x,y)) - 1) + P*(np.e**(Q*r(x,y)**2) - 1) 
    return u

def construct_b(E, X0, Y0, a0):
    def b(x, y):
        return a0 - E + np.arctan2((y - Y0),(x - X0))
    return b

def construct_altura(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
    def z(x, y):
        u = construct_u(C, A, F, V, S, D, P, Q, X0, Y0, a0)
        b = construct_b(E, X0, Y0, a0)
        return np.arccos( np.cos(u(x,y))*np.cos(ep) - np.sin(u(x,y))*np.sin(ep)*np.cos(b(x,y)) )
    return z

def construct_altura_deg(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
    def z(x, y):
        u = construct_u(C, A, F, V, S, D, P, Q, X0, Y0, a0)
        b = construct_b(E, X0, Y0, a0)
        return 90 - np.rad2deg( np.arccos( np.cos(u(x,y))*np.cos(ep) - np.sin(u(x,y))*np.sin(ep)*np.cos(b(x,y)) ) )
    return z

def construct_azimuth(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
    def az(x, y):
        u = construct_u(C, A, F, V, S, D, P, Q, X0, Y0, a0)
        b = construct_b(E, X0, Y0, a0)
        z = construct_altura(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0)
        return np.arcsin( np.sin(b(x,y))*np.sin(u(x,y))/np.sin(z(x,y)) ) + E
    return az

def construct_azimuth_deg(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
    def az(x, y):
        u = construct_u(C, A, F, V, S, D, P, Q, X0, Y0, a0)
        b = construct_b(E, X0, Y0, a0)
        z = construct_altura(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0)
        return np.rad2deg( np.arcsin( np.sin(b(x,y))*np.sin(u(x,y))/np.sin(z(x,y)) ) + E ) + 90
    return az

def construct_alt_xi(x, y, z):
    def xi(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
        alt_teorico = construct_altura_deg(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0)
        return (z - alt_teorico(x, y))**2
    return xi

def construct_az_xi(x, y, z):
    def xi(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0):
        az_teorico = construct_azimuth_deg(C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0)
        return (z - az_teorico(x, y))**2
    return xi

def plot_plano(plano):
    x = np.linspace(0, 1200, 100)
    y = np.linspace(0, 900, 100)
    X, Y = np.meshgrid(x, y)
    Z = plano(X, Y)
    fig,ax=plt.subplots(1,1)
    cp = ax.contourf(X, Y, Z, 10)
    fig.colorbar(cp) # Add a colorbar to a plot
    ax.set_title('Toda una linea que minimiza y=mx+n')
    plt.gca().invert_yaxis()
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    plt.show()

def plot_altura(vector):
    plano = construct_altura_deg(*vector)
    plot_plano(plano)
    
def plot_azimuth(vector):
    plano = construct_azimuth_deg(*vector)
    plot_plano(plano)

In [34]:
%matplotlib qt
param = [1, 0, 0, 0.003, 0.0001, 0.00001, 0, 0, 0, 0.001, 724, 472, 0]
plot_altura(param_5)
plot_azimuth(param_5)

In [33]:
def construct_acumulate_alt_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0 = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_alt_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
#         pbar.update()
        return suma_xi
    return acumulate_xi

def construct_acumulate_az_xi(x_list, y_list, z_list, ctes = None):
    def acumulate_xi(params):
        C, A, F, V, S, D, P, Q, E, ep, X0, Y0, a0 = params
        suma_xi = 0
        for x, y, z in zip(x_list, y_list, z_list):
            xi = construct_az_xi(int(x), int(y), int(z))
            suma_xi += xi(C, A, F, V, S, D, P, Q, E, ep, float(X0), float(Y0), a0)
#         pbar.update()
        return suma_xi
    return acumulate_xi

data = pd.read_csv("Data.pixtab", sep=" ")
data = data.dropna()
# data = drop_outlayers_by_borovicka(data, construct_altura_deg(*ctes[:3],*init_params, *ctes[3:]), 5)
x_exp = data.x.values
y_exp = data.y.values
alt_catalogo = data.alt.values
az_catalogo = np.rad2deg(np.arccos(np.cos(np.deg2rad(data.az.values))))
mega_alt_xi = construct_acumulate_alt_xi(x_exp, y_exp, alt_catalogo)
mega_az_xi = construct_acumulate_az_xi(x_exp, y_exp, az_catalogo)

param_1 = [ 1.08232e+00, -2.94000e-04 , 5.16000e-05 , 3.00000e-03 , 1.00000e-04,
  1.00000e-05,  0.00000e+00 , 0.00000e+00, -9.24000e-05,  1.08232e-03,
  7.24000e+02,  4.72000e+02 , 0.00000e+00]

param_2 = [ 1.12910886e+00,  2.83699436e-03 , 1.62385935e-02 , 3.00235088e-03,
  1.00167129e-04,  1.00729745e-05 , 1.78968400e-05 , 1.87863268e-05,
 -1.10622227e-03 ,-1.23802454e-03  ,7.28955587e+02 , 4.77963259e+02,
  7.76832139e-01]

param_3 = [ 1.12840603e+00 , 2.28744941e-03, -6.51562582e-02,  3.00445129e-03,
 -1.38070859e-04 , 1.47936214e-05 , 4.76105580e-05, -1.02802622e-05,
 -4.95192868e-03, -1.23802454e-03 , 7.28608947e+02 , 4.77611290e+02,
  7.73928718e-01]

param_4 = [ 1.12428482e+00 , 4.24656648e-03, -1.17541302e+00 , 3.00967397e-03,
  5.80256527e-04 ,-2.39030459e-04 , 4.19911841e-03, -6.01712122e-06,
 -4.90153576e-03, -1.23802454e-03 , 7.28624994e+02,  4.77610089e+02,
  7.73978261e-01]

param_5 = [ 1.12830285e+00,  4.18049889e-03, -6.79165404e-01 , 3.13480598e-03,
  2.15742967e-03 , 7.40732289e-04 , 8.39575408e-02, -7.21502667e-06,
 -4.92018266e-03, -1.23802454e-03  ,7.28617422e+02 , 4.77603775e+02,
  7.73955862e-01]


print("alt: ",mega_alt_xi(param_4)/len(x_exp), mega_alt_xi(param_5)/len(x_exp))
print("az: ",mega_az_xi(param_4)/len(x_exp), mega_az_xi(param_5)/len(x_exp))

alt:  2.3028688691218226 2.2084983916027627
az:  15.744627552509371 15.743932495382785
